In [1]:
import nltk
from nltk.corpus import udhr
import keras
from nltk import FreqDist
from gensim.models.word2vec import Word2Vec
import pandas as pd
import regex
import nltk.tokenize.casual
import tensorflow as tf

In [3]:
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.keras.utils.get_file(
    f'{model_name}.zip',
    f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
    cache_dir='.', cache_subdir='', extract=True
)

Exception: URL fetch failure on https://storage.googleapis.com/download.tensorflow.org/models/ted_hrlr_translate_pt_en_converter.zip: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)

In [2]:
# https://stackoverflow.com/questions/39263929/how-can-i-read-tar-gz-file-using-pandas-read-csv-with-gzip-compression-option
df = pd.read_csv('books_labels.tar.gz', compression='gzip', header=0, sep=',', quotechar='"', on_bad_lines='skip')

In [3]:
# https://stackoverflow.com/questions/17424182/extracting-all-rows-from-pandas-dataframe-that-have-certain-value-in-a-specific
spanish_df = df[df['language'] == 'SPA']['text']
spanish_list = spanish_df.to_list() # https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://ioflood.com/blog/dataframe-to-list-pandas/%23:~:text%3DIt%2520is%2520utilized%2520with%2520the,tolist()%2520.%26text%3DIn%2520the%2520example%2520above%252C%2520we,convert%2520it%2520into%2520a%2520list.&ved=2ahUKEwiP3deMnLSFAxVD4ckDHUo7BsEQFnoECA4QAw&usg=AOvVaw3GgfIVdsVo9Dxul02uata1

In [4]:
spanish_list_clean = [regex.sub(r'\([a-zA-z0-9]\)', '', item) for item in spanish_list]
spanish_string = ''.join(spanish_list_clean)
spanish_list_clean = spanish_string.split('.')
spanish_list_clean = [item.strip(' ') for item in spanish_list_clean]
spanish_corpus = spanish_list_clean[:100]

In [5]:
# https://stackoverflow.com/questions/17424182/extracting-all-rows-from-pandas-dataframe-that-have-certain-value-in-a-specific
portuguese_df = df[df['language'] == 'POR']['text']
portuguese_list = portuguese_df.to_list()

In [6]:
# portuguese_list_clean = [regex.sub(r'\([a-zA-z0-9]\)', '', item) for item in portuguese_list]
# print(portuguese_list_clean[:50])
for i, item in enumerate(portuguese_list):
    if type(item) == float:
        portuguese_list.pop(i)
portuguese_string = ''.join(portuguese_list)
portuguese_list = portuguese_string.split('.')
portuguese_list = [item.strip(' ') for item in portuguese_list]
print(portuguese_list[:50])
portuguese_corpus = portuguese_list[:100]

['No segundo ano do rei Dario, no sexto mês, no primeiro dia do mês, veio a palavra do Senhor, por intermédio do profeta Ageu, a Zorobabel, governador de Judá, filho de Sealtiel, e a Josué, o sumo sacerdote, filho de Jeozadaque, dizendo:Assim fala o Senhor dos exércitos, dizendo: Este povo diz: Não veio ainda o tempo, o tempo de se edificar a casa do Senhor', 'Veio, pois, a palavra do Senhor, por intermédio do profeta Ageu, dizendo:Acaso é tempo de habitardes nas vossas casas forradas, enquanto esta casa fica desolada?Ora pois, assim diz o Senhor dos exércitos: Considerai os vossos caminhos', 'Tendes semeado muito, e recolhido pouco; comeis, mas não vos fartais; bebeis, mas não vos saciais; vestis-vos, mas ninguém se aquece; e o que recebe salário, recebe-o para o meter num saco furado', 'Assim diz o Senhor dos exércitos: Considerai os vossos caminhos', 'Subi ao monte, e trazei madeira, e edificai a casa; e dela me deleitarei, e serei glorificado, diz o Senhor', 'Esperastes o muito, ma

In [7]:
# New Dataset: https://huggingface.co/datasets/versae/bibles
spanish_input_texts, spanish_target_texts = [], []
portuguese_input_texts, portuguese_target_texts = [], []
spanish_vocabulary = set()
portuguese_vocabulary = set()
start_token = '[START]'
stop_token = '[END]'
max_training_samples = min(25000, len(spanish_corpus) - 1, len(portuguese_corpus) - 1)

for spanish_input_text in spanish_corpus:
    spanish_target_text = start_token + spanish_input_text + stop_token
    spanish_input_texts.append(spanish_input_text)
    spanish_target_texts.append(spanish_target_text)
    for char in spanish_target_text:
        if char not in spanish_vocabulary:
            spanish_vocabulary.add(char)

for portuguese_input_text in portuguese_corpus:
    if type(portuguese_input_text) == float:
        print(portuguese_input_text)
    portuguese_target_text = start_token + portuguese_input_text + stop_token
    portuguese_input_texts.append(portuguese_input_text)
    portuguese_target_texts.append(portuguese_target_text)
    for char in portuguese_target_text:
        if char not in portuguese_vocabulary:
            portuguese_vocabulary.add(char)

spanish_vocabulary = sorted(spanish_vocabulary)
portuguese_vocabulary = sorted(portuguese_vocabulary)

spanish_vocab_size = len(spanish_vocabulary)
portuguese_vocab_size = len(portuguese_vocabulary)
max_spanish_seq_length = max([len(txt) for txt in spanish_target_texts])
max_portuguese_seq_length = max([len(txt) for txt in portuguese_target_texts])

spanish_token_index = dict([(char, i) for i, char in
                          enumerate(spanish_vocabulary)])
portuguese_token_index = dict([(char, i) for i, char in
                          enumerate(portuguese_vocabulary)])
reverse_spanish_char_index = dict([(i, char) for i, char in
                          spanish_token_index.items()])
reverse_portuguese_char_index = dict([(i, char) for i, char in
                          portuguese_token_index.items()])

import numpy as np

# encoder_input_data = np.zeros((len(spanish_input_texts) + len(portuguese_input_texts),
#                                max(max_spanish_seq_length, max_portuguese_seq_length), spanish_vocab_size + portuguese_vocab_size),
#                                dtype='float32')

# Update with both
spanish_encoder_input_data = np.zeros((len(spanish_input_texts),
                               max_spanish_seq_length, spanish_vocab_size),
                               dtype='float32')
spanish_decoder_input_data = np.zeros((len(spanish_input_texts),
                               max_spanish_seq_length, spanish_vocab_size),
                               dtype='float32')
spanish_decoder_target_data = np.zeros((len(spanish_input_texts),
                               max_spanish_seq_length, spanish_vocab_size),
                               dtype='float32')

for i, (input_text, target_text) in enumerate(zip(spanish_input_texts, spanish_target_texts)):
    for t, char in enumerate(input_text):
        spanish_encoder_input_data[
            i, t, spanish_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        spanish_decoder_input_data[
            i, t, spanish_token_index[char]] = 1.
        if t > 0:
            spanish_decoder_target_data[i, t - 1, spanish_token_index[char]] = 1

In [8]:
batch_size = 64
epochs = 3
num_neurons = 256

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense


spanish_encoder_inputs = Input(shape=(None, spanish_vocab_size))
spanish_encoder = LSTM(num_neurons, return_state=True)
spanish_encoder_outputs, s_state_h, s_state_c = spanish_encoder(spanish_encoder_inputs)
spanish_encoder_states = (s_state_h, s_state_c)

spanish_decoder_inputs = Input(shape=(None, spanish_vocab_size))
spanish_decoder_lstm = LSTM(
    num_neurons, return_sequences=True, return_state=True)
spanish_decoder_outputs, _, _ = spanish_decoder_lstm(
    spanish_decoder_inputs, initial_state=spanish_encoder_states)
spanish_decoder_dense = Dense(
    spanish_vocab_size, activation='softmax')
spanish_decoder_outputs = spanish_decoder_dense(spanish_decoder_outputs)
spanish_spanish_model = Model(
    inputs=[spanish_encoder_inputs, spanish_decoder_inputs],
    outputs=spanish_decoder_outputs)

spanish_spanish_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['acc'])
spanish_spanish_model.fit([spanish_encoder_input_data, spanish_decoder_input_data],
          spanish_decoder_target_data, batch_size=batch_size, epochs=epochs,
          validation_split=0.1)

2024-04-08 21:39:20.151504: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-04-08 21:39:20.151528: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-04-08 21:39:20.151534: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-04-08 21:39:20.151552: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-08 21:39:20.151566: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/3


2024-04-08 21:39:28.116054: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


141/141 ━━━━━━━━━━━━━━━━━━━━ 70s 487ms/step - acc: 0.0495 - loss: 0.4467 - val_acc: 0.0237 - val_loss: 0.5532
Epoch 2/3
141/141 ━━━━━━━━━━━━━━━━━━━━ 67s 475ms/step - acc: 0.0186 - loss: 0.4251 - val_acc: 0.0237 - val_loss: 0.5360
Epoch 3/3
141/141 ━━━━━━━━━━━━━━━━━━━━ 67s 477ms/step - acc: 0.0190 - loss: 0.4215 - val_acc: 0.0237 - val_loss: 0.5332


In [9]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [10]:
from transformer import *
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=spanish_vocab_size,
    target_vocab_size=spanish_vocab_size,
    dropout_rate=dropout_rate)

2024-04-08 23:34:05.552256: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-04-08 23:34:05.552278: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-04-08 23:34:05.552282: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-04-08 23:34:05.552299: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-08 23:34:05.552313: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
output = transformer((spanish_encoder_input_data[0], spanish_decoder_target_data[0]))
print(output.shape)

/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer doe

(520, 67, 67)


In [12]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [13]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [14]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [15]:
ds = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data, spanish_decoder_input_data), spanish_decoder_target_data))
#tf.data.Dataset.bucket_by_sequence_length(ds)

In [16]:
attn_scores = transformer.decoder.dec_layers[-1].last_attn_scores
print(attn_scores.shape)  # (batch, heads, target_seq, input_seq)

(520, 8, 67, 67)


In [17]:
transformer.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder (Encoder)               │ ?                      │     2,647,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Decoder)               │ ?                      │     4,758,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │         8,643 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,414,467 (28.28 MB)

 Trainable params: 7,414,467 (28.28 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
tf.config.run_functions_eagerly(True)
transformer.fit(ds, epochs=epochs)

Epoch 1/3


/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer doe

InvalidArgumentError: {{function_node __wrapped__Sub_device_/job:localhost/replica:0/task:0/device:GPU:0}} Incompatible shapes: [67,128] vs. [0] [Op:Sub]